In [1]:
!pip install piazza-api
!pip install html2text

     ---------------------------------------- 0.0/56.5 kB ? eta -:--:--
     ---------------------------------------- 56.5/56.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33083 sha256=9ce628f06e4a3fb7ce46421bb9ea463998833fe68fd180bae57aa982b0467e8d
  Stored in directory: c:\users\tuyen\appdata\local\pip\cache\wheels\98\7e\0e\dcc9406eea46d4906caf4b5f18a9c26c686602ae82f0387331
Successfully built html2text


In [2]:
import json
import re
import os
import time
from html2text import HTML2Text
from piazza_api import Piazza

HTML_2_TEXT = HTML2Text()
HTML_2_TEXT.ignore_links = True

EMAIL = None
PASSWORD = None
NETWORK_ID = None

if os.path.exists('credentials.json'):
    with open('credentials.json') as f:
        credentials = json.load(f)
        EMAIL = credentials.get('email', None)
        PASSWORD = credentials.get('password', None)
        NETWORK_ID = credentials.get('network_id', None)

def count_good_comments(comment, student_comment_counts, user_id_to_name):
    if (comment['type'] == 'followup' or comment['type'] == 'feedback') and comment['tag_good']:
        student_id = comment.get('uid', None)
        if student_id and student_id not in student_comment_counts:
            student_comment_counts[student_id] = 0
        if student_id:
            student_comment_counts[student_id] += 1
            if student_id not in user_id_to_name:
                user_id_to_name[student_id] = "Unknown"
    for child in comment['children']:
        count_good_comments(child, student_comment_counts, user_id_to_name)

def main():
    piazza = Piazza()
    piazza.user_login(email=EMAIL, password=PASSWORD)
    print("Logged in successfully.")
    network_id = NETWORK_ID
    course = piazza.network(network_id)
    posts = list(course.iter_all_posts(limit=500))  # Convert generator to list
    # for post in posts:
    #     print(post)
    print(f"Found {len(posts)} posts.")

    # Get all users and create a dictionary mapping user IDs to names
    all_users = course.get_all_users()
    print(f"Found {len(all_users)} users.")
    user_id_to_name = {user['id']: user['name'] for user in all_users}
    print(user_id_to_name)

    student_comment_counts = {}

    for post in posts:
        for child in post['children']:
            count_good_comments(child, student_comment_counts, user_id_to_name)
        # time.sleep(1)  # Add a delay between requests

    for student_id, count in student_comment_counts.items():
        student_name = user_id_to_name.get(student_id, "Unknown")
        print(f"Student {student_name} ({student_id}) has {count} helpful comments.")

if __name__ == '__main__':
    main()

Logged in successfully.


RequestError: Could not get post lsu5y3c194d7i6.
Response: {
  "result": null,
  "error_codes": [],
  "error": "Sorry, too fast -- please wait a second and try again.",
  "aid": "luqplqjprx11c7"
}